In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
import statsmodels.formula.api as smf


In [2]:
df = pd.read_csv('data/train_data.csv', delimiter=',')

cols = df.columns[3:-1]
numFeatures = len(cols)

df = df.dropna(subset=cols) #drop na cells 
df = df[df.bedrooms <= 15]
df = df[df.sqft_lot <= 1250000]
df = df[df.sqft_above <= 7000]
df = df[df.sqft_basement <= 3500]
df = df[df.sqft_lot15 <= 500000]

scores = []
for i in df['yr_renovated']:
    if i != 0:
        scores.append((i - 2015) + 82)
    else:
        scores.append(0)
df['renovation_score'] = scores

landvalue = []
view = list(df['view'])
water = list(df['waterfront'])
for i in range(len(water)):
    value = view[i] * 2
    if water[i] == 1:
        value  = value ** 2
    landvalue.append(value)
df['landvalue'] = landvalue

centerpoint= (47.628591, -122.289796)  #center point of city center
dist_df = df.loc[:,['Unique_idx','lat','long','price']]
dist_df['dist_latLon'] = np.sqrt(np.square(dist_df['lat']- centerpoint[0]) + np.square(dist_df['long']-centerpoint[1]))
df['distFromSeattle'] = dist_df['dist_latLon']
# df = df.drop(['lat', 'long'], axis=1)


#202272.96564043706
cols1 = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors','waterfront', 'view', 'condition', 'grade', 'sqft_above','sqft_basement', 'yr_built', 'zipcode', 'lat', 'long', 'yr_renovated', 'sqft_living15', 'sqft_lot15', 'price']

#193483.43463007256
cols2 = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors','landvalue','waterfront', 'view', 'condition', 'grade', 'sqft_above','sqft_basement', 'yr_built', 'distFromSeattle', 'zipcode', 'lat', 'long', 'yr_renovated', 'renovation_score', 'sqft_living15', 'sqft_lot15', 'price']

#200158.15498220024
cols3 = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors','landvalue', 'condition', 'grade', 'sqft_above','sqft_basement', 'yr_built', 'distFromSeattle', 'renovation_score', 'sqft_living15', 'sqft_lot15', 'price']


df_train1 = df.loc[:,cols1]
df_train2 = df.loc[:,cols2]
df_train3 = df.loc[:,cols3]

In [3]:
df_train2.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,landvalue,waterfront,view,condition,grade,...,yr_built,distFromSeattle,zipcode,lat,long,yr_renovated,renovation_score,sqft_living15,sqft_lot15,price
0,3,1.75,2190,7021,1.0,4,0,2,4,7,...,1953,0.137979,98178,47.5033,-122.232,0,0,2180,7155,353000
1,3,2.50,2370,6840,2.0,0,0,0,3,9,...,1987,0.079236,98119,47.6503,-122.366,0,0,1590,4400,300523
2,2,1.00,1230,3800,1.0,0,0,0,3,7,...,1928,0.051157,98115,47.6797,-122.292,0,0,1610,3800,435000
3,4,2.25,2510,9963,1.0,0,0,0,4,9,...,1967,0.117056,98005,47.5973,-122.177,0,0,3110,9963,800000
4,3,1.00,1160,7491,1.0,0,0,0,4,6,...,1917,0.101178,98177,47.7024,-122.359,0,0,1800,2267,417500


In [ ]:
# train_df = pd.read_csv('./data/train_data.csv')
# val_df = pd.read_csv('./data/val_data.csv')
# sample_df = pd.read_csv('./data/sample_submission.csv')
'''
percent = 0.7  #choose percent to split 
n= df_train1.shape[0] #number of rows in data set before splitting
s= math.floor(percent * n)
y= df_train1.iloc[0:s,-1]
X = df_train1.iloc[0:s,:-1]

y_test= df_train1.iloc[s:n,-1]
X_test = df_train1.iloc[s:n,:-1]
'''

In [4]:
# train_df = pd.read_csv('./data/train_data.csv')
# val_df = pd.read_csv('./data/val_data.csv')
# sample_df = pd.read_csv('./data/sample_submission.csv')

percent = 0.7  #choose percent to split 
n= df_train2.shape[0] #number of rows in data set before splitting
s= math.floor(percent * n)
y= df_train2.iloc[0:s,-1]
X = df_train2.iloc[0:s,:-1]

y_test= df_train2.iloc[s:n,-1]
X_test = df_train2.iloc[s:n,:-1]

In [ ]:
# train_df = pd.read_csv('./data/train_data.csv')
# val_df = pd.read_csv('./data/val_data.csv')
# sample_df = pd.read_csv('./data/sample_submission.csv')

'''
percent = 0.7  #choose percent to split 
n= df_train3.shape[0] #number of rows in data set before splitting
s= math.floor(percent * n)
y= df_train3.iloc[0:s,-1]
X = df_train3.iloc[0:s,:-1]

y_test= df_train3.iloc[s:n,-1]
X_test = df_train3.iloc[s:n,:-1]
'''

In [5]:
model=sm.OLS(y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.920
Model:                            OLS   Adj. R-squared (uncentered):              0.920
Method:                 Least Squares   F-statistic:                              6934.
Date:                Wed, 12 Feb 2020   Prob (F-statistic):                        0.00
Time:                        14:33:33   Log-Likelihood:                     -1.6359e+05
No. Observations:               12098   AIC:                                  3.272e+05
Df Residuals:                   12078   BIC:                                  3.274e+05
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
bedrooms         -3.376e+04   2391.158    -14.119      0.000   -3.84e+04   -2.91e+04
bathrooms          2.92e+04   3954.056      7.384      0.000    2.14e+04    3.69e+04
sqft_living        103.3825      2.821     36.644      0.000      97.852     108.913
sqft_lot             0.2180      0.059      3.723      0.000       0.103       0.333
floors           -5482.3127   4334.281     -1.265      0.206    -1.4e+04    3013.573
landvalue         9263.0299   1386.172      6.682      0.000    6545.910     1.2e+04
waterfront         6.69e+04   7.12e+04      0.939      0.348   -7.27e+04    2.06e+05
view              3.208e+04   3914.808      8.196      0.000    2.44e+04    3.98e+04
condition         3.577e+04   2801.224     12.770      0.000    3.03e+04    4.13e+04
grade             8.412e+04   2596.331     32.401      0.000     7.9e+04    8.92e+04
sqft_above          74.9872      2.824     26.556      0.000      69.452      80.522
sqft_basement       28.3952      3.207      8.854      0.000      22.109      34.682
yr_built         -1549.7224     87.598    -17.691      0.000   -1721.429   -1378.016
distFromSeattle  -7.414e+05   2.58e+04    -28.762      0.000   -7.92e+05   -6.91e+05
zipcode             28.2686     28.593      0.989      0.323     -27.778      84.315
lat               3.152e+05   1.66e+04     18.995      0.000    2.83e+05    3.48e+05
long              1.242e+05   1.95e+04      6.370      0.000     8.6e+04    1.62e+05
yr_renovated       -63.7338     17.038     -3.741      0.000     -97.131     -30.337
renovation_score  2967.6012    528.347      5.617      0.000    1931.956    4003.246
sqft_living15       34.8306      4.139      8.416      0.000      26.718      42.943
sqft_lot15          -0.0596      0.089     -0.670      0.503      -0.234       0.115
==============================================================================
Omnibus:                     8569.729   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           441465.917
Skew:                           2.838   Prob(JB):                         0.00
Kurtosis:                      32.044   Cond. No.                     8.78e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.59e-22. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [12]:
X['price']=y  #need to add price as last col for statsmodel
X.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,landvalue,waterfront,view,condition,grade,...,yr_built,distFromSeattle,zipcode,lat,long,yr_renovated,renovation_score,sqft_living15,sqft_lot15,price
0,3,1.75,2190,7021,1.0,4,0,2,4,7,...,1953,0.137979,98178,47.5033,-122.232,0,0,2180,7155,353000
1,3,2.50,2370,6840,2.0,0,0,0,3,9,...,1987,0.079236,98119,47.6503,-122.366,0,0,1590,4400,300523
2,2,1.00,1230,3800,1.0,0,0,0,3,7,...,1928,0.051157,98115,47.6797,-122.292,0,0,1610,3800,435000
3,4,2.25,2510,9963,1.0,0,0,0,4,9,...,1967,0.117056,98005,47.5973,-122.177,0,0,3110,9963,800000
4,3,1.00,1160,7491,1.0,0,0,0,4,6,...,1917,0.101178,98177,47.7024,-122.359,0,0,1800,2267,417500


In [36]:
est03 = smf.ols('price ~ bedrooms + bathrooms + sqft_living + sqft_lot + floors + landvalue + waterfront + view + condition + grade + sqft_above + sqft_basement + yr_built + zipcode + yr_renovated +renovation_score +distFromSeattle + np.power(distFromSeattle, 2)', X).fit()
#cols2 = ['              bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors','landvalue','waterfront', 'view', 'condition', 'grade', 'sqft_above','sqft_basement', 'yr_built', 'distFromSeattle', 'zipcode', 'lat', 'long', 'yr_renovated', 'renovation_score', 'sqft_living15', 'sqft_lot15', 'price']

est03.summary().tables[0]

Dep. Variable:,price,R-squared:,0.744
Model:,OLS,Adj. R-squared:,0.744
Method:,Least Squares,F-statistic:,2069.
Date:,"Wed, 12 Feb 2020",Prob (F-statistic):,0.00
Time:,15:16:24,Log-Likelihood:,-1.6319e+05
No. Observations:,12098,AIC:,3.264e+05
Df Residuals:,12080,BIC:,3.265e+05
Df Model:,17,,
Covariance Type:,nonrobust,,


In [37]:
est03.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,8.005e+07,3.56e+06,22.508,0.000,7.31e+07,8.7e+07
bedrooms,-3.497e+04,2315.462,-15.102,0.000,-3.95e+04,-3.04e+04
bathrooms,2.216e+04,3824.110,5.795,0.000,1.47e+04,2.97e+04
sqft_living,117.1244,2.539,46.129,0.000,112.147,122.101
sqft_lot,0.1988,0.043,4.655,0.000,0.115,0.282
floors,-3.432e+04,4260.955,-8.055,0.000,-4.27e+04,-2.6e+04
landvalue,9434.2803,1338.559,7.048,0.000,6810.489,1.21e+04
waterfront,7.953e+04,6.88e+04,1.157,0.247,-5.53e+04,2.14e+05
view,3.243e+04,3763.363,8.618,0.000,2.51e+04,3.98e+04
condition,2.978e+04,2729.626,10.911,0.000,2.44e+04,3.51e+04


In [39]:
#model_skl= LinearRegression().fit(X.iloc[:,-1],y)  
#print('Rsquared(sklearn) = ', model_skl.score(X.iloc[:,-1],y))
#yhat_skl = model_skl.predict(X_test)
#yhat_skl = yhat_skl.astype(int)
#delta = yhat_skl - y_test
#mse_skl = np.sqrt(np.mean(np.square(delta)))
#print('mse_skl = ', mse_skl)

In [35]:
#compare to test data
yhat = est03.predict(X_test)
yhat = yhat.astype(int)
#print(y_test)
#print(yhat)
#print(np.abs(yhat-y_test))
delta = yhat - y_test
mse = np.sqrt(np.mean(np.square(delta)))
print('mse (stats model) = ', mse)

mse (stats model) =  188615.8641178612


In [16]:
#both stats model and sklearn have almost the as mse
#mse/mse_skl

1.069443812166527

In [40]:
val_df = pd.read_csv('./data/val_data.csv')
sample_df = pd.read_csv('./data/sample_submission.csv')

In [41]:
sample_df.head()

,Unique_idx,price
0,17599,220000
1,5879,241000
2,10654,350000
3,1901,375000
4,19138,276000


In [42]:
val_df.head()

,Unique_idx,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,15544,1310430130,20141009T000000,4,2.75,2790,6600,2.0,0,0,...,9,2790,0,2000,0,98058,47.4362,-122.109,2900,6752
1,17454,2540830020,20150401T000000,3,2.25,1630,6449,1.0,0,0,...,7,1310,320,1986,0,98011,47.7275,-122.232,1620,7429
2,21548,8835770330,20140819T000000,2,1.50,2370,184231,2.0,0,0,...,11,2370,0,2005,0,98045,47.4543,-121.778,3860,151081
3,3427,7732400490,20141105T000000,4,2.50,2270,7665,2.0,0,0,...,9,2270,0,1986,0,98052,47.6612,-122.148,2450,8706
4,8809,2800031,20150401T000000,3,1.00,1430,7599,1.5,0,0,...,6,1010,420,1930,0,98168,47.4783,-122.265,1290,10320
